In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
data = pd.read_csv('/Users/kevins/Documents/CSImpact/data/cleaned_prof_data_v4.csv')

In [3]:
data

,name,affiliation,homepage,scholar_id,citations,h-index
0,P. Vassalos,AUEB,http://pubs.dbs.uni-leipzig.de/aa/aff/Greece/A...,NOSCHOLARPAGE,NaN,NaN
1,Stavros Toumpis,AUEB,https://195.251.255.130/en/faculty/toumpis-sta...,LyiVPKEAAAAJ,2804.0,21.0
2,Ion Androutsopoulos,AUEB,http://www.aueb.gr/users/ion,4UJm5EQAAAAJ,11918.0,42.0
3,Michalis K. Titsias,AUEB,http://www.aueb.gr/users/mtitsias,B-SbkAwAAAAJ,5255.0,30.0
4,Vasilios A. Siris,AUEB,http://www.aueb.gr/users/vsiris,Vj3O0cEAAAAJ,5967.0,31.0
...,...,...,...,...,...,...
20644,Ali C. Begen,Özyeğin University,https://ali.begen.net,ZIZWsXoAAAAJ,5964.0,34.0
20645,Erhan Öztop,Özyeğin University,https://robotics.ozyegin.edu.tr/members/erhan-...,A8YEQMMAAAAJ,3702.0,28.0
20646,Yasar Safkan,Özyeğin University,https://cs.ozyegin.edu.tr/en/faculty/yasarsafkan,NOSCHOLARPAGE,NaN,NaN
20647,Hasan Sözer,Özyeğin University,https://faculty.ozyegin.edu.tr/hsozer,L3FuUX0AAAAJ,847.0,16.0


In [4]:
data = data.sort_values('name')

In [5]:
data = data.reset_index(drop = True)

In [15]:
#data= data.drop(['Unnamed: 0'], axis = 1)

In [6]:
data

,name,affiliation,homepage,scholar_id,citations,h-index
0,A Min Tjoa,TU Wien,http://www.ifs.tuwien.ac.at/tjoa,x8qCMhcAAAAJ,6069.0,40.0
1,A. Akbari Azirani,IUST,http://ce.iust.ac.ir/page.php?slct_pg_id=6537&...,pCil4_cAAAAJ,1326.0,18.0
2,A. Aldo Faisal,Imperial College London,https://www.imperial.ac.uk/people/a.faisal,WjHjbrwAAAAJ,6780.0,31.0
3,A. Antony Franklin,IIT Hyderabad,http://www.iith.ac.in/~antony/index.html,LVfqLuoAAAAJ,NaN,NaN
4,A. B. M. Alim Al Islam,BUET,https://sites.google.com/site/abmalimalislam,K-AIPzQAAAAJ,562.0,11.0
...,...,...,...,...,...,...
20644,Ítalo Cunha,UFMG,https://www.dcc.ufmg.br/~cunha,bTAky1EAAAAJ,2036.0,23.0
20645,Óscar Cánovas,University of Murcia,http://webs.um.es/ocanovas/miwiki/doku.php,iAPkCb8AAAAJ,625.0,13.0
20646,Özgür Erkent,Hacettepe University,https://web.cs.hacettepe.edu.tr/~ozgurerkent,5QMAbisAAAAJ,348.0,11.0
20647,Özgür Ulusoy,Bilkent University,http://www.cs.bilkent.edu.tr/~oulusoy,M_vU9pQAAAAJ,4716.0,37.0


In [11]:
earliest_paper_years = []
#total_rows = data.shape[0]
test_data = data.head(100)
total_test_rows = test_data.shape[0]

for i in range(total_test_rows):
    s_id = test_data.iloc[i,3]
    if not pd.isna(test_data.iloc[i,4]):
        html_text = requests.get(f'https://scholar.google.com/citations?hl=en&user={s_id}&view_op=list_works&sortby=pubdate').text
        soup = BeautifulSoup(html_text, 'lxml')
        values = soup.find_all('button', id = 'gsc_bpf_more')

        driver.get(f'https://scholar.google.com/citations?hl=en&user={s_id}&view_op=list_works&sortby=pubdate')
        text1 = driver.page_source
        while True:
            try:
                button = driver.find_element_by_id('gsc_bpf_more')
                time.sleep(0.75)
                button.click()
            except ElementNotInteractableException:
                final_text = driver.page_source
                break
            except NoSuchElementException:
                driver.get(f'https://scholar.google.com/citations?hl=en&user={s_id}&view_op=list_works&sortby=pubdate')
                break
            text2=driver.page_source
            if text1 == text2:
                final_text = text2
                break
            else:
                text1 = text2

        final_soup = BeautifulSoup(final_text, 'lxml')
        years = final_soup.find_all('td', class_ = 'gsc_a_y')
        years_text = []
        for year in years:
            years_text.append(year.text)

        clean_years = [year_text for year_text in years_text if year_text != '']

        min_y = int(clean_years[0])
        for clean_y in clean_years:
            if int(clean_y) < min_y:
                min_y = int(clean_y)
        earliest_paper_years.append(str(min_y))

    else:
        earliest_paper_years.append('')

/var/folders/sw/rc481v691q5883ld6bm6jzq80000gn/T/ipykernel_69164/2416858570.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  button = driver.find_element_by_id('gsc_bpf_more')


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=103.0.5060.134)
Stacktrace:
0   chromedriver                        0x00000001009cdef9 chromedriver + 4480761
1   chromedriver                        0x00000001009595d3 chromedriver + 4003283
2   chromedriver                        0x00000001005ec338 chromedriver + 410424
3   chromedriver                        0x00000001005dc80f chromedriver + 346127
4   chromedriver                        0x00000001005ddcb2 chromedriver + 351410
5   chromedriver                        0x00000001005d64dc chromedriver + 320732
6   chromedriver                        0x00000001005ed752 chromedriver + 415570
7   chromedriver                        0x0000000100651bab chromedriver + 826283
8   chromedriver                        0x000000010063f953 chromedriver + 751955
9   chromedriver                        0x0000000100615cd5 chromedriver + 580821
10  chromedriver                        0x0000000100616d25 chr

In [3]:
from selenium import webdriver
import webbrowser
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import time
from selenium import webdriver
from selenium.webdriver.safari.options import Options
from fake_useragent import UserAgent

In [10]:

options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.implicitly_wait(random() * 10)

[WDM] - ====== WebDriver manager ======


2022-07-20 15:51:27,871 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-20 15:51:28,049 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


2022-07-20 15:51:28,051 INFO Get LATEST chromedriver version for 103.0.5060 google-chrome


[WDM] - Driver [/Users/kevins/.wdm/drivers/chromedriver/mac64/103.0.5060.134/chromedriver] found in cache


2022-07-20 15:51:28,178 INFO Driver [/Users/kevins/.wdm/drivers/chromedriver/mac64/103.0.5060.134/chromedriver] found in cache


/var/folders/sw/rc481v691q5883ld6bm6jzq80000gn/T/ipykernel_69164/2287171904.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [116]:
#
#
#
#TEST#####
#
#

#1. get html text, make soup
#2. find button
#3. check if button is disabled
#4a. Yes: find earliest paper
#4b No: Click button, go to #3

html_text = requests.get('https://scholar.google.com/citations?hl=en&user=C1skWKgAAAAJ&view_op=list_works&sortby=pubdate').text
soup = BeautifulSoup(html_text, 'lxml')
values = soup.find_all('button', id = 'gsc_bpf_more')

driver.get('https://scholar.google.com/citations?hl=en&user=C1skWKgAAAAJ&view_op=list_works&sortby=pubdate')
text1 = driver.page_source
while True:
    button = driver.find_element_by_id('gsc_bpf_more')
    button.click()
    text2=driver.page_source
    if text1 == text2:
        final_text = text2
        break
    else:
        text1 = text2
        
final_soup = BeautifulSoup(final_text, 'lxml')
years = final_soup.find_all('td', class_ = 'gsc_a_y')
years_text = []
for year in years:
    years_text.append(year.text)

clean_years = [year_text for year_text in years_text if year_text != '']

min_y = int(clean_years[0])
for clean_y in clean_years:
    if int(clean_y) < min_y:
        min_y = int(clean_y)



/var/folders/sw/rc481v691q5883ld6bm6jzq80000gn/T/ipykernel_89736/623857051.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  button = driver.find_element_by_id('gsc_bpf_more')


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

options = Options()
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Safari(=options)


Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


TypeError: __init__() got an unexpected keyword argument 'safari_options'

In [7]:
pip install webdriver

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
ERROR: Could not find a version that satisfies the requirement webdriver (from versions: none)
ERROR: No matching distribution found for webdriver
Note: you may need to restart the kernel to use updated packages.


In [9]:
from cmath import isnan
import os
from random import random
import time
from typing import Any
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from csv import writer
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.implicitly_wait(random() * 10)